In [2]:
import os
import jax.numpy as jnp
import shutil

### Part 3: Trajectory Evaluation and g2o

_Refer to the example notebooks for installation instructions_

### Evo

So you've implemented 2D SLAM, great! Now, what? We need a measure of how good the trajectory is. The error/loss used earlier doesn't tell us much about how the trajectory differs from the ground truth. Here, we try to do just this - compute error metrics. Rather than computing these from scratch, we will just Evo - https://github.com/MichaelGrupp/evo/.

Look at the absolute pose error (APE) and relative pose error (RPE). What do they capture and how are they calculated (descriptive answer)? How do these metrics differ in methodology? Can we determine if the error is more along the x/y axis?

Answer the above questions and report errors for the obtained trajectory.

In [15]:
# Answer the above questions. Also include plots/images.
def getEvo(infile,gtFile):
    os.system('evo_rpe kitti ' + gtFile + ' ' + infile + ' -v --plot --plot_mode xy')
    os.system('evo_ape kitti ' + gtFile + ' ' + infile + ' -v --plot --plot_mode xy')
    os.system('evo_traj kitti ' + gtFile + ' ' + infile + ' -v --plot --plot_mode xy')

In [16]:
os.system('python3 ../misc/g2o_to_kitti.py ../data/out.g2o ../data/out.kitti')
getEvo('../data/out.kitti', '../data/gt.kitti')

Figure(640x480)
saved '../data/out.kitti' from '../data/out.g2o'
--------------------------------------------------------------------------------
Loaded 120 poses from: ../data/gt.kitti
Loaded 120 poses from: ../data/out.kitti
--------------------------------------------------------------------------------
Found 119 pairs with delta 1 (frames) among 120 poses using consecutive pairs.
Compared 119 relative pose pairs, delta = 1 (frames) with consecutive pairs.
Calculating RPE for translation part pose relation...
--------------------------------------------------------------------------------
RPE w.r.t. translation part (m)
for delta = 1 (frames) using consecutive pairs
(not aligned)

       max	0.287002
      mean	0.117280
    median	0.113936
       min	0.005656
      rmse	0.132108
       sse	2.076860
       std	0.060812

--------------------------------------------------------------------------------
Plotting results... 


/home/rohan/miniconda3/envs/MR/lib/python3.8/site-packages/evo/tools/plot.py:165: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


--------------------------------------------------------------------------------
Loaded 120 poses from: ../data/gt.kitti
Loaded 120 poses from: ../data/out.kitti
--------------------------------------------------------------------------------
Compared 120 absolute pose pairs.
Calculating APE for translation part pose relation...
--------------------------------------------------------------------------------
APE w.r.t. translation part (m)
(not aligned)

       max	10.234741
      mean	5.312775
    median	5.655704
       min	0.000000
      rmse	6.185583
       sse	4591.372590
       std	3.167943

--------------------------------------------------------------------------------
Plotting results... 


/home/rohan/miniconda3/envs/MR/lib/python3.8/site-packages/evo/tools/plot.py:165: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


--------------------------------------------------------------------------------
Loaded 120 poses from: ../data/gt.kitti
Loaded 120 poses from: ../data/out.kitti
--------------------------------------------------------------------------------
name:	gt
infos:
	nr. of poses	120
	path length (m)	52.976517997999295
	pos_end (m)	[-2.8 -4.5  0. ]
	pos_start (m)	[-8.  5.  0.]
--------------------------------------------------------------------------------
name:	out
infos:
	nr. of poses	120
	path length (m)	53.762411967921594
	pos_end (m)	[-0.15280963  4.0175176   0.        ]
	pos_start (m)	[-8.  5.  0.]


/home/rohan/miniconda3/envs/MR/lib/python3.8/site-packages/evo/tools/plot.py:165: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


Try to play around with this tool and add any other plots that you think might be relevant/interesting.

**APE**: Absolute Pose Error - it is used to calculate the direct error between the poses of the 2 files we provide to it. It is useful as it helps us obtain a wider view of how different the actual poses are in the ground truth frame and the optimized frame, giving us an overview of how correct our poses are. However, before comparing the direct poses, we need to align them, as they may be in different coordinate frames. This can be achieved using Horn's method, which finds the rigid-body transformation S.

The error matrix at time `i` is defined as:
$$E_{i} := Q_{i}^{-1} S P_{i}$$
and the APE is defined as RMSE from the error matrices over all the time indices:

$$APE_{rmse} = (\frac{1}{n} \sum_{i = 1}^{n} ||trans(E_{i})||^{2})$$

The mean and median values are computed. Actually, APE is the average deviation from the ground trajectory per frame.


**RPE**: Relative Pose Error - Compares pose deltas instead of the absolute poses. It checks how the transformations of each pose differ from each other and how the values drift over time.

The relative pose error matrix at time `i` is defined as:
$$F_{i}^{\Delta} := (Q_{i}^{-1}Q_{i + \Delta})^{-1}(P_{i}^{-1}P_{i + \Delta})$$
We get $m = n - \Delta$ matrices from $n$ camera poses.

Even within drift, there are 2 different components - rotational and translational
$$RPE_{rot}^{i, \Delta} = \frac{1}{m}\sum_{i = 1}^{m}\angle(rot(F_{i}^{\Delta}))$$
$$RPE_{trans}^{i, \Delta} = (\frac{1}{m}\sum_{i = 1}^{m} ||trans(F_{i})||^{2})^{\frac{1}{2}}$$

**Difference**: `APE` measures the absolute error in the corresponding poses, while `RPE` measures the accumulation of drift in the transformations between the current and previous poses.

**Determining axis of higher error**: From the given graphs, we cannot see the difference in X and Y's respective APE and RPE values since the graph gives us APE and RPE wrt index. When it comes to RPE, it makes no sense for there to be differing X and Y errors, since its calculating the relative drift in poses, and that will differ in rotation/translation not x and y (in fact, comparing over X or Y will basically ignore the rotational part of the error). We can, however, calculate the APE over X and Y separately for each pose (calculate $\Delta x$ and $\Delta y$ separately) and compare. 

### g2o

Install g2o as mentioned in `examples/g2o.ipynb` and optimise `edges.txt`, the file you used earlier. Also use `g2o_viewer` and optimize `intel` (a trajectory in the Intel research lab) and `sphere`. They should look something like:


<table><tr>
<td> <img src="../misc/intel.jpg" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="../misc/sphere.jpg" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>

Write briefly about your observations and try out few options in the GUI. What do they do, how do they perform?

In [8]:
# Your answer
# shutil.copyfile('../data/edges.txt', '../data/edges.g2o')
cmd = "g2o -o ../data/opt.g2o ../data/unOpt.g2o"
os.system(cmd)

os.system("python3 ../misc/g2o_to_kitti.py ../data/opt.g2o ../data/opt.kitti")
getEvo('../data/opt.kitti', '../data/gt.kitti')

Read input from ../data/unOpt.g2o
Loaded 120 vertices
Loaded 139 edges
# graph is fixed by priors or already fixed vertex
Initial chi2 = 573824.084635
saving ../data/opt.g2o ... done.


Figure(640x480)
saved '../data/opt.kitti' from '../data/opt.g2o'
--------------------------------------------------------------------------------
Loaded 120 poses from: ../data/gt.kitti
Loaded 120 poses from: ../data/opt.kitti
--------------------------------------------------------------------------------
Found 119 pairs with delta 1 (frames) among 120 poses using consecutive pairs.
Compared 119 relative pose pairs, delta = 1 (frames) with consecutive pairs.
Calculating RPE for translation part pose relation...
--------------------------------------------------------------------------------
RPE w.r.t. translation part (m)
for delta = 1 (frames) using consecutive pairs
(not aligned)

       max	0.256198
      mean	0.114314
    median	0.097856
       min	0.009129
      rmse	0.128960
       sse	1.979061
       std	0.059691

--------------------------------------------------------------------------------
Plotting results... 


/home/rohan/miniconda3/envs/MR/lib/python3.8/site-packages/evo/tools/plot.py:165: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


--------------------------------------------------------------------------------
Loaded 120 poses from: ../data/gt.kitti
Loaded 120 poses from: ../data/opt.kitti
--------------------------------------------------------------------------------
Compared 120 absolute pose pairs.
Calculating APE for translation part pose relation...
--------------------------------------------------------------------------------
APE w.r.t. translation part (m)
(not aligned)

       max	1.863155
      mean	0.704550
    median	0.558657
       min	0.000000
      rmse	0.882165
       sse	93.385719
       std	0.530871

--------------------------------------------------------------------------------
Plotting results... 


/home/rohan/miniconda3/envs/MR/lib/python3.8/site-packages/evo/tools/plot.py:165: UserWarning: The figure layout has changed to tight
  fig.tight_layout()


--------------------------------------------------------------------------------
Loaded 120 poses from: ../data/gt.kitti
Loaded 120 poses from: ../data/opt.kitti
--------------------------------------------------------------------------------
name:	gt
infos:
	nr. of poses	120
	path length (m)	52.976517997999295
	pos_end (m)	[-2.8 -4.5  0. ]
	pos_start (m)	[-8.  5.  0.]
--------------------------------------------------------------------------------
name:	opt
infos:
	nr. of poses	120
	path length (m)	53.81648854770469
	pos_end (m)	[-3.28756 -5.45456  0.     ]
	pos_start (m)	[-8.  5.  0.]


/home/rohan/miniconda3/envs/MR/lib/python3.8/site-packages/evo/tools/plot.py:165: UserWarning: The figure layout has changed to tight
  fig.tight_layout()
